# Data Mining Project - Bank

## Project Developed by:
- *Duarte Sardão*
- *Gabriel Ferreira*
- *Miguel Lopes*
- *Sérgio Estêvão*

## Table of Contents
1. Business Understanding

## Business Understanding

The dataset is composed of a series of information that describe the activity of a Czech bank during the 90s. This includes account, client, credit cards, transaction and loan information as well as information regarding the districts where the bank's clients reside.

With this information we are expected to create a predictive model that allows the end user to determine whether or not a client is suitable to get a loan. 

In this business context, a client is granted a loan if the model predicts that the client will be able to pay it in full.

Taking into account that an unpaid loan can result in up to 100% loss, while interest is unlikely to account for 100% profit over the loan value, an unpaid loan incurrs much higher negative value than a loan that was not granted. Thus in this case, our goal should be to minimize unpaid loans and thus, as relates to data analysis: minimize the number of false positives.

## Data Treatment

In [4]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

We start by loading the data into dataframes and removing information that has no relation to loans, as that data is irrelevant to the problem being targeted in this project.

In [5]:
# loan_id;account_id;date;amount;duration;payments;status
loans = pd.read_csv("./data/loan_dev.csv", sep=";",dtype=int)
transactions = pd.read_csv("./data/trans_dev.csv", sep=";",dtype={"trans_id":int,"account_id":int,"date":int,"type":str,"operation":str,"amount":float,"balance":float,"k_symbol":str,"bank":str,"account":str})
accounts = pd.read_csv("./data/account.csv", sep=";",dtype={"account_id":int,"district_id":int,"frequency":str,"date":int})
cards = pd.read_csv("./data/card_dev.csv", sep=";",dtype={"card_id":int,"disp_id":int,"type":str,"issued":int})
clients = pd.read_csv("./data/client.csv", sep=";",dtype=int)
dispositions = pd.read_csv("./data/disp.csv", sep=";",dtype={"disp_id":int,"client_id":int,"account_id":int,"type":str})
districts = pd.read_csv("./data/district.csv", na_values=['?'], sep=";",dtype={"code":int, "name":str, "region":str,"no. of inhabitants":int,"no. of municipalities with inhabitants < 499":int,"no. of municipalities with inhabitants 500-1999":int,"no. of municipalities with inhabitants 2000-9999":int, "no. of municipalities with inhabitants >10000": int, "no. of cities":int, "ratio of urban inhabitants":float, "average salary":float, "unemploymant rate '95":float, "unemploymant rate '96":float, "no. of enterpreneurs per 1000 inhabitants": float, "no. of commited crimes '95":int, "no. of commited crimes '96":int})

print("\n==== Before Clean-up ====")
print("Transactions: ", len(transactions))
print("Accounts: ", len(accounts))
print("Dispositions: ", len(dispositions))
print("Clients: ", len(clients))
print("Cards: ", len(cards))
print("Districts: ", len(districts))

#Clean up information not associated with loans

account_ids = loans['account_id'].unique()

transactions.query("account_id in @account_ids", inplace=True)
accounts.query("account_id in @account_ids", inplace=True)
dispositions.query("account_id in @account_ids", inplace=True)

#save how many clients are associated to an account, might matter
accounts['num_clients'] = accounts.apply(lambda row: dispositions['account_id'].value_counts()[row.account_id], axis=1)




#but drop their info
dispositions.query("type == 'OWNER'", inplace=True)

client_ids = dispositions['client_id'].unique()
disp_ids = dispositions['disp_id'].unique()
district_ids = accounts['district_id'].unique()

#rename cuz of ending spaces
districts = districts.rename(columns={"code ": "code",
                   "name ": "name"})

clients.query("client_id in @client_ids", inplace=True)
cards.query("disp_id in @disp_ids", inplace=True)
districts.query("code in @district_ids", inplace=True)

print("\n==== After Clean-up ====")
print("Transactions: ", len(transactions))
print("Accounts: ", len(accounts))
print("Dispositions: ", len(dispositions))
print("Clients: ", len(clients))
print("Cards: ", len(cards))
print("Districts: ", len(districts))


==== Before Clean-up ====
Transactions:  396685
Accounts:  4500
Dispositions:  5369
Clients:  5369
Cards:  177
Districts:  77

==== After Clean-up ====
Transactions:  24494
Accounts:  328
Dispositions:  328
Clients:  328
Cards:  11
Districts:  73


Next we start to create a unified data-frame. Starting by merging the Disposition and Accounts data-frames into the Clients dataset.

In [6]:
from datetime import datetime
import math

dispositions_accounts = pd.merge(dispositions, accounts, on="account_id")
dispositions_accounts.drop(columns="type", inplace=True)
dispositions_accounts = dispositions_accounts.rename(columns={"date": "creation", "district_id": "branch_district_id"})
current_date = datetime.strptime("1997-01-01", '%Y-%m-%d')
dispositions_accounts["creation_date"] = pd.to_datetime(dispositions_accounts["creation"], format="%y%m%d")
dispositions_accounts["account_age"] = dispositions_accounts["creation_date"].map(lambda date: math.floor((current_date - date).days/365.2425))
dispositions_accounts.pop("creation_date")
clients = pd.merge(dispositions_accounts, clients, on="client_id")
clients = clients.rename(columns={"district_id": "client_district_id"})
clients.head()

,disp_id,client_id,account_id,branch_district_id,frequency,creation,num_clients,account_age,birth_number,client_district_id
0,2,2,2,1,monthly issuance,930226,2,3,450204,1
1,25,25,19,21,monthly issuance,950407,1,1,395423,21
2,78,78,67,16,monthly issuance,941019,1,2,445613,16
3,158,158,132,40,issuance after transaction,960511,2,0,455703,40
4,210,210,173,66,monthly issuance,931126,2,3,391130,66


After that, and still on the Clients dataset, we parse the birth_number which contains information about the clients birth date and sex. Also we calculate the age of the clients considering "1997-01-01" as the current time.

In [7]:
from datetime import datetime
import math
#transform birth number into birthdate and sex
birth_number = clients["birth_number"]
clients["sex"] = birth_number.map(lambda nr: "Male" if nr % 10000 < 5000 else "Female").astype('category')
clients["birthdate"] = birth_number.map(lambda nr: str(nr) if nr % 10000 < 5000 else str(nr-5000))
clients["birthdate"] = pd.to_datetime("19"+clients["birthdate"], format='%Y%m%d')

current_date = datetime.strptime("1997-01-01", '%Y-%m-%d')
birthdates = clients["birthdate"]
clients["age"] = birthdates.map(lambda dt: math.floor((current_date - dt).days/365.2425))

clients.drop(columns="birth_number", inplace=True)
clients.head()


,disp_id,client_id,account_id,branch_district_id,frequency,creation,num_clients,account_age,client_district_id,sex,birthdate,age
0,2,2,2,1,monthly issuance,930226,2,3,1,Male,1945-02-04,51
1,25,25,19,21,monthly issuance,950407,1,1,21,Female,1939-04-23,57
2,78,78,67,16,monthly issuance,941019,1,2,16,Female,1944-06-13,52
3,158,158,132,40,issuance after transaction,960511,2,0,40,Female,1945-07-03,51
4,210,210,173,66,monthly issuance,931126,2,3,66,Male,1939-11-30,57


We then parse other dates, converting them from the provided format into the YYYY-MM-DD format

In [8]:
#other date treatment (yymmdd string to datetime format)
cards["issued"] = cards["issued"].map(lambda nr: str(nr))
cards["issued"] = pd.to_datetime("19"+cards["issued"], format='%Y%m%d')

transactions["date"] = transactions["date"].map(lambda nr: str(nr))
transactions["date"] = pd.to_datetime("19"+transactions["date"], format='%Y%m%d')

loans["date_datetime"] = pd.to_datetime(loans["date"], format="%y%m%d")
loans["loan_age"] = loans["date_datetime"].map(lambda date: math.floor((current_date - date).days/365.2425))
loans.pop("date_datetime")

loans["date"] = loans["date"].map(lambda nr: str(nr))
loans["date"] = pd.to_datetime("19" + loans["date"], format='%Y%m%d')


clients["creation"] = clients["creation"].map(lambda nr: str(nr))
clients["creation"] = pd.to_datetime("19"+clients["creation"], format='%Y%m%d')


We then go on to merge the card info into the Clients dataset

In [9]:
clients = pd.merge(clients, cards, on="disp_id", how="outer")
clients.drop(columns="card_id", inplace=True)
clients = clients.rename(columns={"type": "card_type","issued": "card_issue"})
clients.head()

,disp_id,client_id,account_id,branch_district_id,frequency,creation,num_clients,account_age,client_district_id,sex,birthdate,age,card_type,card_issue
0,2,2,2,1,monthly issuance,1993-02-26,2,3,1,Male,1945-02-04,51,NaN,NaT
1,25,25,19,21,monthly issuance,1995-04-07,1,1,21,Female,1939-04-23,57,NaN,NaT
2,78,78,67,16,monthly issuance,1994-10-19,1,2,16,Female,1944-06-13,52,NaN,NaT
3,158,158,132,40,issuance after transaction,1996-05-11,2,0,40,Female,1945-07-03,51,NaN,NaT
4,210,210,173,66,monthly issuance,1993-11-26,2,3,66,Male,1939-11-30,57,NaN,NaT


here were only checking correlations of numeric values: it shouldnt matter at this point but conver male/female to 0-1, frequency, dates etc?

In [10]:
#check correlations for transactions
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = transactions.corr()
corr.style.background_gradient(cmap='coolwarm')

/tmp/ipykernel_420443/3320867235.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = transactions.corr()


,trans_id,account_id,amount,balance
trans_id,1.000000,0.730371,-0.187568,0.005802
account_id,0.730371,1.000000,0.012000,0.048992
amount,-0.187568,0.012000,1.000000,0.483082
balance,0.005802,0.048992,0.483082,1.000000


No major correlations: mostly between transaction ID's and account ID's as many transactions are unique for an account.

In [11]:
#check correlations for clients
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = clients.corr()
corr.style.background_gradient(cmap='coolwarm')

/tmp/ipykernel_420443/325381064.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = clients.corr()


,disp_id,client_id,account_id,branch_district_id,num_clients,account_age,client_district_id,age
disp_id,1.000000,0.999787,0.999983,-0.003577,0.059016,0.037368,0.052161,-0.115264
client_id,0.999787,1.000000,0.999885,-0.003647,0.060825,0.037600,0.052514,-0.113043
account_id,0.999983,0.999885,1.000000,-0.003619,0.059682,0.037495,0.052254,-0.114610
branch_district_id,-0.003577,-0.003647,-0.003619,1.000000,0.002584,-0.067807,0.793821,-0.098763
num_clients,0.059016,0.060825,0.059682,0.002584,1.000000,-0.020142,0.010364,-0.050117
account_age,0.037368,0.037600,0.037495,-0.067807,-0.020142,1.000000,-0.025120,-0.001424
client_district_id,0.052161,0.052514,0.052254,0.793821,0.010364,-0.025120,1.000000,-0.078435
age,-0.115264,-0.113043,-0.114610,-0.098763,-0.050117,-0.001424,-0.078435,1.000000


Very high correlations with associated ID's (we've already done merge on the associated dispositions). Significant correlation between client district id and and branch district id as in most cases they are the same.

In [12]:
#check correlations for districts
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = districts.corr()
corr.style.background_gradient(cmap='coolwarm')

/tmp/ipykernel_420443/84124200.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = districts.corr()


,code,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96
code,1.000000,0.019858,-0.186831,0.340731,0.318589,0.270787,0.018381,0.022253,-0.250335,0.404195,0.419104,-0.468978,-0.108782,-0.131782
no. of inhabitants,0.019858,1.000000,-0.312467,-0.202190,-0.106207,0.037061,-0.278867,0.463214,0.640440,-0.113837,-0.137509,0.264523,0.978436,0.973129
no. of municipalities with inhabitants < 499,-0.186831,-0.312467,1.000000,0.217136,-0.157365,-0.224381,0.071168,-0.444359,-0.353844,-0.177005,-0.210463,-0.065477,-0.314270,-0.285687
no. of municipalities with inhabitants 500-1999,0.340731,-0.202190,0.217136,1.000000,0.577600,0.066485,0.351456,-0.639956,-0.407941,-0.042703,-0.092587,-0.089230,-0.299030,-0.281315
no. of municipalities with inhabitants 2000-9999,0.318589,-0.106207,-0.157365,0.577600,1.000000,0.032620,0.506122,-0.361284,-0.181635,-0.072288,-0.080391,-0.142781,-0.212998,-0.209699
no. of municipalities with inhabitants >10000,0.270787,0.037061,-0.224381,0.066485,0.032620,1.000000,0.261026,0.285589,-0.032217,0.435900,0.411644,-0.347598,-0.029992,-0.034472
no. of cities,0.018381,-0.278867,0.071168,0.351456,0.506122,0.261026,1.000000,-0.062416,-0.279502,-0.095152,-0.133049,-0.063659,-0.325615,-0.307510
ratio of urban inhabitants,0.022253,0.463214,-0.444359,-0.639956,-0.361284,0.285589,-0.062416,1.000000,0.607056,0.190784,0.183726,0.000102,0.450710,0.423680
average salary,-0.250335,0.640440,-0.353844,-0.407941,-0.181635,-0.032217,-0.279502,0.607056,1.000000,-0.068265,-0.096442,0.262970,0.653193,0.634418
unemploymant rate '95,0.404195,-0.113837,-0.177005,-0.042703,-0.072288,0.435900,-0.095152,0.190784,-0.068265,1.000000,0.980877,-0.556197,-0.143342,-0.153745


Conclusions about correlation for districts:

* Data about sucessive years have very high correlation -> Save only one year and extract a value for growth

* Absolute values (no of crimes) has very high correlation with population -> Turn into crimerate with division

Since some values related to the number of crimes committed in 95 and 96 were missing, we also use the K-Nearest Neighbor algorithm to fill in empty values in the newly created crime growth column and unemployment growth column.

### Removing redudant fields

In [13]:
#calc crimerates rather than whole values
districts['crimerate95'] = districts.apply(lambda row: row["no. of commited crimes '95 "] / row["no. of inhabitants"], axis=1)
districts['crimerate96'] = districts.apply(lambda row: row["no. of commited crimes '96 "] / row["no. of inhabitants"], axis=1)
#calculate with growth rate from 95-96
districts['crimegrowth'] = districts.apply(lambda row: row["crimerate96"] / row["crimerate95"], axis=1)
districts['unempgrowth'] = districts.apply(lambda row: row["unemploymant rate '96 "] / row["unemploymant rate '95 "], axis=1)
 
#drops columns now calculated with relative vals and growth
districts.drop(columns="crimerate95", inplace=True)
districts.drop(columns="no. of commited crimes '95 ", inplace=True)
districts.drop(columns="no. of commited crimes '96 ", inplace=True)
districts.drop(columns="unemploymant rate '95 ", inplace=True)
 
#clean up names for better experience
districts = districts.rename(columns={"no. of inhabitants": "population",
                   "no. of municipalities with inhabitants < 499 ": "munip<499",
                   "no. of municipalities with inhabitants 500-1999": "munip500-1999",
                   "no. of municipalities with inhabitants 2000-9999 ": "munip2000-9999",
                   "no. of municipalities with inhabitants >10000 ": "munip>10000",
                   "no. of municipalities with inhabitants >10000 ": "munip>10000",
                   "no. of cities ": "cities",
                   "ratio of urban inhabitants ": "urbanization",
                   "average salary ": "avg_salary",
                   "unemploymant rate '96 ": "unemployment96",
                   "no. of enterpreneurs per 1000 inhabitants ": "entrepreneurs/1000"
                  })
 
#handle missing vals remaining in crimegrowth, unempgrowth with K-nearest neighbour
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
df_knn = districts.copy()
df_knn.drop(columns=["code", "name", "region"], inplace=True)
#scale values from 0-1 for KNN
scaler = MinMaxScaler(feature_range=(0, 1))
df_knn = pd.DataFrame(scaler.fit_transform(df_knn), columns = df_knn.columns)
#target values scaled, reset
df_knn['crimegrowth'] = districts['crimegrowth']
df_knn['unempgrowth'] = districts['unempgrowth']
#impute knn
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_knn), columns=df_knn.columns)
 
districts.fillna(df_knn_imputed, inplace=True)


ModuleNotFoundError: No module named 'sklearn'

### Data Extraction

Moving on to transactions, we combine transactions of each client by determining their accounts balance at 4 moments in time (the most recent balance, the balance 1, 6 and 12 months before that most recent date)

In [ ]:
transaction_index = {} #['account_id] = {RecentBalance, MonthBalance, 3MonthBalance 6MonthBalance, 12MonthBalance, ...type of transaction...}

transactions_bydate = transactions.sort_values(by="date", ascending=False)



for index, transaction in transactions_bydate.iterrows():
    account_id = transaction["account_id"]
    if account_id not in transaction_index.keys():
        transaction_index[account_id] = {}
        transaction_index[account_id]["num_salaries"] = 0
        transaction_index[account_id]["average_salary"] = 0
        transaction_index[account_id]["last_amount"] =  transaction["balance"]
        transaction_index[account_id]["recent_balance"] = transaction
        transaction_index[account_id]["household_payments"] = 0
        transaction_index[account_id]["payment_for_statement_counter"] = 0
        transaction_index[account_id]["insurrance_payments"] = 0       
        transaction_index[account_id]["sanction_payment_counter"] = 0
        transaction_index[account_id]["old_age_pensions"] = 0


    else:

    
        delta = transaction_index[account_id]["recent_balance"]["date"] - transaction["date"]
        #print(type(delta))

        if delta.days > 30*(transaction_index[account_id]["num_salaries"]+1):
          prev_avg_salary = transaction_index[account_id]["average_salary"] 
          num_salaries = transaction_index[account_id]["num_salaries"]
          new_salary = transaction_index[account_id]["last_amount"] - transaction["balance"]

          transaction_index[account_id]["average_salary"] = (prev_avg_salary*num_salaries + new_salary)/(num_salaries+1)
          transaction_index[account_id]["last_amount"] = transaction["balance"]
          transaction_index[account_id]["num_salaries"] += 1

          #Alternative way to get monthly balance of all months
          #month_balance_str = str(transaction_index[account_id]["num_salaries"]) + "_month_balance"
          #transaction_index[account_id][month_balance_str] = transaction["balance"]

        #Alternative way of getting the monthly balance of specific months
        if "month_balance" not in transaction_index[account_id].keys():
            if delta.days > 30:
                transaction_index[account_id]["month_balance"] = transaction["balance"]
        elif "3_month_balance" not in transaction_index[account_id].keys():
            if delta.days > 90: 
              transaction_index[account_id]["3_month_balance"] = transaction["balance"]
        elif "6_month_balance" not in transaction_index[account_id].keys():
            if delta.days > 180:
                transaction_index[account_id]["6_month_balance"] = transaction["balance"]
        elif "12_month_balance" not in transaction_index[account_id].keys():
            if delta.days > 365: 
                transaction_index[account_id]["12_month_balance"] = transaction["balance"]

    if transaction["k_symbol"] == "household":
      transaction_index[account_id]["household_payments"] += 1
    elif transaction["k_symbol"] == "payment for statement":
      transaction_index[account_id]["payment_for_statement_counter"] += 1
    elif transaction["k_symbol"] == "insurrance payment":
      transaction_index[account_id]["insurrance_payments"] += 1  
    elif transaction["k_symbol"] == "sanction interest if negative balance":
      transaction_index[account_id]["sanction_payment_counter"] += 1
    elif transaction["k_symbol"] == "old-age pension":
      transaction_index[account_id]["old_age_pensions"] += 1


for index in transaction_index.keys():
    transaction_index[index]["recent_balance"] = transaction_index[index]["recent_balance"]["balance"]


We then merge the account balances into the Clients dataset

In [ ]:

#merge the balance dataset (transaction_index) with the accounts dataset to get the final accounts dataset with balance information
for account_id in transaction_index.keys():
    transaction_index[account_id]["account_id"] = int(account_id)

transaction_index_df = pd.DataFrame(transaction_index)

transaction_index_df = transaction_index_df.transpose()
transaction_index_df.pop("last_amount")



new_clients = pd.merge(clients, transaction_index_df, on="account_id", how="outer")
new_clients
new_clients.to_csv('./new_clients.csv', index = False, header=True)


Next, we merge the Loans and Clients datasets and add 3 new columns: the age of the client at the time of the loan, the age of the account at the time of the loan and the age of the card at the time of the loan. These columns are derived from the subtraction of the birth date, creation date and card issue date columns from the date of the loan column, respectively, and allow us to determine correlation between these values and the other columns of the merged dataset.

In [ ]:
#assuming all data treated at this point
loans = pd.merge(loans, new_clients, on="account_id", how="outer")
loans = pd.merge(loans, districts, left_on='client_district_id', right_on='code')
loans = pd.merge(loans, districts, left_on='branch_district_id', right_on='code', suffixes=('', '_branch'))
loans['age_when_loan'] = loans.apply(lambda row: (row['date'] - row['birthdate']), axis=1)
loans['age_when_loan'] = loans['age_when_loan'].dt.days
loans['account_age_when_loan'] = loans.apply(lambda row: (row['date'] - row['creation']), axis=1)
loans['account_age_when_loan'] = loans['account_age_when_loan'].dt.days
loans['card_age_when_loan'] = loans.apply(lambda row: (row['date'] - row['card_issue']), axis=1)
loans['card_age_when_loan'] = loans['card_age_when_loan'].dt.days



Next, we convert all string columns to integer columns so they can be used by the classification algorithms.

In [ ]:
columns = ["frequency", "sex", "name", "region", "name_branch", "region_branch"]
for col in columns:
  values = list(pd.unique(loans[col]))
  loans[col] = loans[col].map(lambda name: values.index(name))

Finally, we drop the unnecassery columns and export the final dataframe.

In [ ]:
#drop ids used for merge
#coluns dropped because of too many missing values in rows: card_type, card_issue, card_age_when_loan
loans.drop(columns=["date", "account_id", "disp_id", "client_id", "branch_district_id", "client_district_id", "creation","birthdate", "card_type", "card_issue", "card_age_when_loan"], inplace=True)

import seaborn as sns

sns.heatmap(loans.corr());


loans.to_csv('./final_data.csv', index = False, header=True)

### Outlier Removal

Now that we have a dataset that combines all the initial datasets, we define a function for removing outliers, based on 1.5x inter-quartile rule. After that we apply it to the amount and duration columns - isto tem de ser melhorado (?)

In [ ]:

def remove_outliers(df,column):
    Q1 = np.percentile(df[column], 25,
                    method = 'midpoint')
    
    Q3 = np.percentile(df[column], 75,
                    method = 'midpoint')
    IQR = Q3 - Q1
    # Upper bound
    upper = np.where(df[column] >= (Q3+1.5*IQR))
    # Lower bound
    lower = np.where(df[column] <= (Q1-1.5*IQR))
    df.drop(upper[0], inplace = True)
    df.drop(lower[0], inplace = True)


remove_outliers(loans,'amount')
remove_outliers(loans,'duration')


TypeError: ignored

In [ ]:
#check correlations for loans
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = loans.corr()
corr.style.background_gradient(cmap='coolwarm')

Final treatment, drop nans


In [ ]:
# removed strings, dates and columns with high number of NaN values
data=loans.drop(columns=['date','creation','birthdate','frequency','sex','card_type','card_issue','name_branch','region_branch','name','region','card_age_when_loan','6_month_balance','12_month_balance'])

# removing rows with NaN values
print("Before NaN removal:"+str(len(data)))

data.dropna(axis=0,inplace=True)

print("After NaN removal:"+str(len(data)))

labels=data.loc[:,"status"]


Begin classification algo here

In [ ]:
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

#### Undersampling

Copiei as cenas do miguel com os imports movidos mas acho que em teoria depois do downsampling copiaria-se a estrutura que está no smote (sem a parte do smote)

In [ ]:
minority=data[data['status']==-1]
majority=data[data['status']==1]

from sklearn.utils import resample

data_majority_down_sampled= resample(majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(minority),     # to match minority class
                                 random_state=0)

data_down_sampled = data_majority_down_sampled.append(minority)


data.drop(columns=['status'],inplace=True)


labels_down_sampled=data_down_sampled.loc[:,"status"]
data_down_sampled.drop(columns=['status'],inplace=True)

labels_down_sampled,data_down_sampled=shuffle(labels_down_sampled,data_down_sampled,random_state=0)


x_train=data_down_sampled[0:len(data_down_sampled)//2]
y_train=labels_down_sampled[0:len(data_down_sampled)//2]


x_test = pd.merge(data, data_down_sampled, on=['loan_id'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
y_test=labels_down_sampled[len(data_down_sampled)//2:]

pipe = make_pipeline(StandardScaler(), LogisticRegression())


pipe.fit(x_train, y_train)
predictions = pipe.predict(x_test)



logistic_regr_score = pipe.score(x_test, y_test)
f1_score_results=f1_score(y_test,predictions,average='weighted')


print("\nLogistic Regression Score: " + str(logistic_regr_score))
print("\nLogistic Regression F1 Score: " + str(f1_score_results))

pipe = make_pipeline(StandardScaler(), RandomForestClassifier())

pipe.fit(x_train, y_train)
predictions = pipe.predict(x_test)

random_forest_score = pipe.score(x_test, y_test)
print("\nRandom Forest Score: " + str(random_forest_score))

pipe = make_pipeline(StandardScaler(), SVC())

pipe.fit(x_train, y_train)
predictions = pipe.predict(x_test)

svm_score = pipe.score(x_test, y_test)
print("\nSVM Score: " + str(svm_score))

pipe = make_pipeline(StandardScaler(), GaussianNB())

pipe.fit(x_train, y_train)
predictions = pipe.predict(x_test)

nb_score = pipe.score(x_test, y_test)
print("\nNB Score: " + str(nb_score))


#### Smote

In [ ]:
splits = 5
cv = StratifiedKFold(n_splits=splits)
log_reg_score = 0
fold = 0
for train_idx, test_idx, in cv.split(data, labels):
    fold += 1
    X_train, y_train = data[train_idx], labels[train_idx]
    X_test, y_test = data[test_idx], labels[test_idx]
    X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    model = make_pipeline(StandardScaler(), LogisticRegression())
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)
    print(f'For fold {fold}:')
    print(f' Logistic Regression Accuracy: {model.score(X_test, y_test)}')
    f1_score_fold = f1_score(y_test, y_pred)
    print(f'Logistic Regression f-score: {f1_score_fold}')
    log_reg_score += f1_score_fold

print(f'Avg f-score for logistic regression: {log_reg_score / splits}')

random_forest_score = 0
fold = 0
for train_idx, test_idx, in cv.split(data, labels):
    fold += 1
    X_train, y_train = data[train_idx], labels[train_idx]
    X_test, y_test = data[test_idx], labels[test_idx]
    X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    model = make_pipeline(StandardScaler(), RandomForestClassifier())
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)
    print(f'For fold {fold}:')
    print(f'Random Forest Accuracy: {model.score(X_test, y_test)}')
    f1_score_fold = f1_score(y_test, y_pred)
    print(f'Random Forest f-score: {f1_score_fold}')
    random_forest_score += f1_score_fold

print(f'Avg f-score for random forest: {random_forest_score / splits}')

svm_score = 0
fold = 0
for train_idx, test_idx, in cv.split(data, labels):
    fold += 1
    X_train, y_train = data[train_idx], labels[train_idx]
    X_test, y_test = data[test_idx], labels[test_idx]
    X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    model = make_pipeline(StandardScaler(), SVC())
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)
    print(f'For fold {fold}:')
    print(f'SVM Accuracy: {model.score(X_test, y_test)}')
    f1_score_fold = f1_score(y_test, y_pred)
    print(f'SVM f-score: {f1_score_fold}')
    svm_score += f1_score_fold

print(f'Avg f-score for SVM: {svm_score / splits}')

nb_score = 0
fold = 0
for train_idx, test_idx, in cv.split(data, labels):
    fold += 1
    X_train, y_train = data[train_idx], labels[train_idx]
    X_test, y_test = data[test_idx], labels[test_idx]
    X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    model = make_pipeline(StandardScaler(), GaussianNB())
    model.fit(X_train, y_train)  
    y_pred = model.predict(X_test)
    print(f'For fold {fold}:')
    print(f'Naive Bayes Accuracy: {model.score(X_test, y_test)}')
    f1_score_fold = f1_score(y_test, y_pred)
    print(f'Naive Bayes f-score: {f1_score_fold}')
    nb_score += f1_score_fold

print(f'Avg f-score for naive bayes: {nb_score / splits}')

poderia-se já fazer undersampling/smote-kfold-hypertuning de todos os modelos de uma vez mas acho que se calhar será melhor ver os que apresentam melhores resultados, pegar tipo em 2/3 e fazer adicionar hypertuning a esses (dps tmb dá pra comparar e ver a melhoria)